<a href="https://colab.research.google.com/github/afiaka87/clip-guided-diffusion/blob/main/colab_clip_guided_diff_hq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generates images from text prompts with CLIP guided diffusion.

By Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses OpenAI's 256x256 unconditional ImageNet diffusion model (https://github.com/openai/guided-diffusion) together with CLIP (https://github.com/openai/CLIP) to connect text prompts with images. 

adjustments to notebook by afiaka87 (https://github.com/afiaka87)

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@title Download the pretrained 256px diffusion checkpoint. Rerun if download fails.

%cd /content

%mkdir /content/checkpoints

!wget --continue 'https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt'

In [ ]:
from IPython.display import clear_output
#@title Install dependencies
%pip install clip-anytorch
# !git clone https://github.com/openai/CLIP

!git clone https://github.com/openai/guided-diffusion
# !pip install -e ./CLIP
!pip install -e ./guided-diffusion
%pip install kornia clip-anytorch

clear_output()

In [ ]:
#@title Imports
import os
import math
import sys
from pathlib import Path

from IPython import display
from PIL import Image
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm

sys.path.append('./CLIP')
sys.path.append('./guided-diffusion')

import clip
from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults
import kornia.augmentation as K

In [ ]:
#@title `MakeCutouts` implementation

# used by everything except for channel and grasycale
# augment_chance = 0.6 #@param {type: 'number'}
# channel_chance = 0.1  #@param {type: 'number'}
# grayscale_chance = 0.1 #@param {type: 'number'}
# K.RandomMotionBlur(3, 15, 0.5, p=0.25),
# K.RandomThinPlateSpline(p=0.25),
# K.RandomAffine(degrees=7, p=0.25, padding_mode="border"),
# K.RandomSolarize(0.01, 0.01, p=0.25),
# K.RandomSharpness(p=1.75),
# K.RandomChannelShuffle(p=0.25),

custom_augment_list = [ ]

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.5):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow 
        self.augs = nn.Sequential()

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutout = F.interpolate(cutout, (self.cut_size, self.cut_size),
                                   mode='bilinear', align_corners=False)
            cutouts.append(cutout)
        return self.augs(torch.cat(cutouts))

In [ ]:
#@title Utility Functions
def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)


def tv_loss(input):
    """L2 total variation loss, as in Mahendran et al."""
    input = F.pad(input, (0, 1, 0, 1), 'replicate')
    x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
    y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
    return (x_diff**2 + y_diff**2).mean([1, 2, 3])

# from github.com/mehdidc/feed_forward_vqgan_clip
def tokenize_lines(line_separated, out="tokenized.pkl"):
    """Save each line of `line_separated` as a CLIP embed. Save CLIP embeds to single file for usage later on. """
    texts = line_separated.splitlines()
    T = clip.tokenize(texts, truncate=True)
    torch.save(T, out)

def tokenize_imagenet():
    !wget --continue --quiet "https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt" 
    imgnet_idx_lbl = open('imagenet1000_clsidx_to_labels.txt').read().splitlines()
    clean_captions = []
    for idx_label in imgnet_idx_lbl:
        imgnet_lbl = re.sub(r'\d+: ', '', line).replace(" '","").replace(" '","").replace("  ", " ").replace("'", '').split(',') # get rid of 'digit: ' then get rid of weird spaces then ' then ,
        if imgnt_lbl[1] is '':
            clean_captions.append(imgnt_lbl[0])
        else:
            clean_captions.append(imgnt_lbl[1])

 
def imagenet_class_line_to_caption():
    !wget --continue --quiet "https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt" 
    imagenet_class_labels = open('imagenet1000_clsidx_to_labels.txt').read().splitlines()
    line = random.choice(imagenet_class_labels)
    labels_in_line = re.sub(r'\d+: ', '', line).replace(" '","").replace(" '","").replace("  ", " ").replace("'", '').split(',') # get rid of 'digit: ' then get rid of weird spaces then ' then ,
    print("prompt will be ignored and a random class will be chosen instead. Set `random_imagenet_class` to False and re-run this cell if that is not desired.")
    if labels_in_line[1] is '':
        return labels_in_line[0]
    return " ".join(labels_in_line)

def load_guided_diffusion(
    diffusion_steps=None,
    timestep_respacing=None,
    device=None,
    class_cond=False,
    rescale_timesteps=True,
):
    assert device is not None, "device must be set"
    model_config = model_and_diffusion_defaults()
    model_config.update({
        "attention_resolutions": "32, 16, 8",
        "class_cond": class_cond,
        "diffusion_steps": diffusion_steps,
        "rescale_timesteps": rescale_timesteps,
        "timestep_respacing": timestep_respacing,
        "image_size": 256,
        "learn_sigma": True,
        "noise_schedule": "linear",
        "num_channels": 256,
        "num_head_channels": 64,
        "num_res_blocks": 2,
        "resblock_updown": True,
        "use_fp16": True,
        "use_scale_shift_norm": True,
    })
    model, diffusion = create_model_and_diffusion(**model_config)
    model.load_state_dict(torch.load("/content/256x256_diffusion_uncond.pt", map_location="cpu"))
    model.requires_grad_(False).eval().to(device)
    for name, param in model.named_parameters():
        if "qkv" in name or "norm" in name or "proj" in name:
            param.requires_grad_()
    if model_config["use_fp16"]:
        model.convert_to_fp16()
    return model, diffusion

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def parse_prompt(prompt):                                               # NR: Weights after colons
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])


In [ ]:
#@title Load `guided-diffusion` and `clip` models
#@markdown `diffusion_steps` - Total number of steps for diffusion. Increase for slower runtime but greater quality.
diffusion_steps =  1000#@param {type: 'integer'}
clip_model_name = 'ViT-B/16' #@param ["ViT-B/16", "ViT-B/32", "RN50", "RN101", "RN50x4", "RN50x16"] 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#@markdown `timestep_respacing` - less than or equal to `diffusion_steps`.  
#@markdown Map sampling to a lower number of steps. Decrease for faster runtimes with decrease in quality. 
#@markdown (optional) use `ddim` sampling.
timestep_respacing = "1000" #@param ["25", "50", "100", "250", "500", "1000", "ddim25", "ddim100", "ddim250", "ddim500"] {allow-input: true}

# Load guided-diffusion model
model, diffusion = load_guided_diffusion(
    diffusion_steps=diffusion_steps,
    timestep_respacing=timestep_respacing,
    device=device,
    class_cond=False,
    rescale_timesteps=True,
)
#@markdown - `ViT-B/32` is quite good.
#@markdown - `RN50x16` uses a ton of VRAM and is only slightly better than `ViT-B/16`
#@markdown - `ViT-B/16` is a tad slower but higher quality.

clip_model = clip.load(clip_model_name, jit=False)[0].eval().requires_grad_(False).to(device)
clip_size = clip_model.visual.input_resolution
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                 std=[0.26862954, 0.26130258, 0.27577711])


print(f"Using device: {device}")

In [ ]:
#@title Settings for the run
prompt = 'Armchair but it looks more look an avocado really. the armchair is imitating the look of an avocado. Armchair avocado is on display.' #@param {type: 'string'}
#@markdown `Or...`
random_imagenet_class = False #@param {type: 'boolean'}
if random_imagenet_class is True:
    prompt = imagenet_class_line_to_caption()
#@markdown `img_prompts` - comma separated string with paths of images to guide diffusion with
#@markdown - `img1.png,img2.png,img3.png`
img_prompts = "" #@param {type: "string"}
img_prompts = img_prompts.split(",")
#@markdown `img_prompt_weight` - may need to increase or decrease for `img_prompts` to work well.
img_prompt_weight =  0#@param {type: 'integer'} 
print(len(img_prompts))
#@markdown `cutn` increasing seems to help for certain prompts but has diminishing returns for many. Uses more VRAM.
cutn = 64#@param {type: 'integer'}
batch_size = 1#@param {type: 'integer'}

text_prompt_weight = 1000 #@param {type: 'integer'}
tv_weight = 95 #@param {type: 'integer'}

seed = 0 #@param {type: 'integer'}

image_size = 256 # Cant change for now

print(f"Using prompt: '{prompt}'")
print(f'Using image prompt/s: {img_prompts}')
print(f"batch size: {batch_size}, text_prompt_weight: {text_prompt_weight}, img_prompt_weight: {img_prompt_weight}, tv_scale: {tv_weight}, cutn: {cutn}, seed: {seed}")

In [ ]:
#@title Actually do the run
prefix_path = '/content/outputs'
num_cutouts = cutn
save_scrollback = False #@param {type: "boolean"}
save_frequency =  25#@param {type: "integer"}
if seed is not None:
    torch.manual_seed(seed)

text_embed = clip_model.encode_text(clip.tokenize(prompt).to(device)).float()

make_cutouts = MakeCutouts(clip_size, num_cutouts)

cur_t = diffusion.num_timesteps - 1


# Convert images to tensors
img_tensors = []

if img_prompts[0] != '':
    for img_prompt in img_prompts:
        img_path = Path(img_prompt)
        assert img_path.exists(), f"Image {img_prompt} does not exist."
        img = resize_image(Image.open(img_path).convert("RGB"), (image_size, image_size))
        img_tensors.append(TF.to_tensor(img))

print(f'Found {len(img_tensors)} images to use as guides for generation.')

# Customize guided-diffusion model with function that uses CLIP guidance.
def cond_fn(x, t, y=None):
    with torch.enable_grad():
        x = x.detach().requires_grad_()
        n = x.shape[0]
        my_t = torch.ones([n], device=device, dtype=torch.long) * current_timestep
        out = diffusion.p_mean_variance(
            model, x, my_t, clip_denoised=False, model_kwargs={"y": y}
        )
        fac = diffusion.sqrt_one_minus_alphas_cumprod[current_timestep]
        x_in = out["pred_xstart"] * fac + x * (1 - fac)

        make_cutouts = MakeCutouts(clip_size, num_cutouts)
        clip_in = normalize(make_cutouts(x_in.add(1).div(2)))
        cutout_embeds = clip_model.encode_image(clip_in).float().view([num_cutouts, n, -1])

        # spherical distance between diffusion cutouts and text to loss
        text_prompt_spherical_loss = spherical_dist_loss(cutout_embeds, text_embed.unsqueeze(0)).mean(0).sum()
        text_prompt_spherical_loss = text_prompt_spherical_loss.mean() * text_prompt_weight

        img_prompt_spherical_loss = torch.zeros([num_cutouts, n], device=device)
        # spherical distance between diffusion cutouts and image to loss
        if len(img_tensors) > 0:
            for img in img_tensors:
                img_prompt_cutouts = normalize(make_cutouts(img.unsqueeze(0).to(device)))
                img_clip_embed = clip_model.encode_image(img_prompt_cutouts).float().view([num_cutouts, n, -1])
                img_prompt_spherical_loss += spherical_dist_loss(cutout_embeds, img_clip_embed.unsqueeze(0)).mean(0).sum()
            img_prompt_spherical_loss /= len(img_tensors) 
            img_prompt_spherical_loss = img_prompt_spherical_loss.mean() * img_prompt_weight

        # denoising loss
        tv_denoise_loss = tv_loss(x_in).sum() * tv_weight

        loss = text_prompt_spherical_loss + img_prompt_spherical_loss.sum() + tv_denoise_loss
        # print(f"Text loss: {text_prompt_spherical_loss}")
        # print(f"Image loss: {img_prompt_spherical_loss.sum()}")
        # print(f"Denoise loss: {tv_denoise_loss}")
        # print("---")
        # print(f"Total loss: {loss}")
        return -torch.autograd.grad(loss, x)[0]
 
if timestep_respacing.startswith("ddim"):
    diffusion_sample_loop = diffusion.ddim_sample_loop_progressive
else:
    diffusion_sample_loop = diffusion.p_sample_loop_progressive

samples = diffusion_sample_loop(
    model,
    (batch_size, 3, image_size, image_size),
    clip_denoised=False,
    model_kwargs={},
    cond_fn=cond_fn,
    progress=True,
)
%rm progress_*
print(f"Attempting to generate the caption:")
print(prompt)
try:
    current_timestep = diffusion.num_timesteps - 1
    for step, sample in enumerate(samples):
        current_timestep -= 1
        if step % save_frequency == 0 or current_timestep == -1:
            for j, image in enumerate(sample["pred_xstart"]):
                if not save_scrollback:
                    clear_output(wait=True)
                filename = os.path.join(
                    prefix_path, f"batch_idx_{j:04}_iteration_{step:04}.png"
                )
                TF.to_pil_image(image.add(1).div(2).clamp(0, 1)).save(filename)
                tqdm.write(f"Step {step}, output {j}:")
                display.display(display.Image(filename))
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print(f"CUDA OOM error occurred. Lower the batch_size or num_cutouts and try again.")
    else:
        raise e

In [ ]:
#@title Generate Video (WIP) - coding knowledge required

# comment out this line first

!ffmpeg -framerate 5 -pattern_type glob -i '/content/outputs/*.png' -c:v libx264 -pix_fmt yuv420p "progress_final.mp4"